In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
import os

In [15]:
actions = np.array(['hola', 'j', 'no'])
#Add abs path to desired folder
DATA_PATH = os.path.abspath("")
no_sequences = 30
sequence_length = 30
start_folder = 30

In [16]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            if res.shape != (63,):
                continue
            window.append(res)
        
        if len(window) != sequence_length:
            continue
        
        sequences.append(window)
        labels.append(label_map[action])

sequences = np.array(sequences)
print(f"Shape of sequences: {sequences.shape}")
print(f"Shape of labels: {np.array(labels).shape}")

In [25]:
np.array(sequences).shape
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [27]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [28]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,63)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [29]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
EPOCHS_NUM = 180
model.fit(X_train, y_train, epochs=EPOCHS_NUM, callbacks=[tb_callback])

In [ ]:
model.summary()

In [ ]:
model.save('action.h5')